## Compressive Strength prediction model

In [1]:
import os

import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
# Import train_test_split function
from sklearn.model_selection import train_test_split, KFold, cross_val_score

In [2]:
g25 = pd.read_excel(r'E:\Prasad\IITM\HACKETHONS\concastle IIT indore\DATA SET\Mix Design Report G25.xlsx','Mix Designs')
g30 = pd.read_excel(r'E:\Prasad\IITM\HACKETHONS\concastle IIT indore\DATA SET\Mix Design Report G30.xlsx','Mix Designs')

In [3]:
g30

,No.,Date,Location,Type of Coarse Aggregate,Type of Fine Aggregate,Max. Size of Coarse Aggregate (mm),Passing 0.6mm Sieve(%),Target Mean Strength (N/mm2),Cement O.P.C (Kg/m3),W/C Ratio,Water Content (Kg/m3),Additive Type & Dos. (/100 Kg of cement),Total Aggregate (Kg/m3),Fine Aggregate (Kg/m3),Coarse Aggregate (Kg/m3),Workability Slump (mm),Hardened Concrete Desnity (avg.),Strength (N/mm2) (7 days avg.),Strength (N/mm2) (28 days avg.),Comments
0,515,2014-07-04 00:00:00,NaN,Crushed,Natural,20.0,NaN,43.0,375,0.48,180,Type JY-4 pump Aiding * Dos 1.25l lit,1885,715,1070,180.0,2442.0,37.6,45.8,NaN
1,133,2013-04-04 00:00:00,NaN,Natural,Natural,40.0,NaN,43.0,375,0.48,180,NaN,1825,730,1095,110.0,2384.0,25.1,32.8,NaN
2,528,14/7/2014,NaN,Natural,Natural,40.0,NaN,43.0,385,0.47,180,Type SP901 * Dos 1.0 lit,1825,730,1095,230.0,2395.0,39.2,47.1,NaN
3,438,2013-09-06 00:00:00,NaN,Crushed,Natural,20.0,NaN,43.0,365,0.51,185,Type CF555 * Dos. 1.0 lit,1900,780,1120,180.0,2457.0,30.7,45.8,NaN
4,428,2013-09-15 00:00:00,NaN,Crushed,Natural,20.0,NaN,43.0,365,0.51,185,Type CF555 * Dos. 1.0 lit,1900,780,1120,180.0,2455.0,27.0,30.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,540,2015-04-02 00:00:00,NaN,Natural,Natural,40.0,17.0,43.0,375,0.44,165,Type SP901 * Dos 1.0 lit,1860,690,1170,165.0,2405.0,30.4,NaN,NaN
570,194,2016-10-31 00:00:00,NaN,Natural,Natural,40.0,14.9,43.0,375,0.48,180,NaN,1845,700,1140,100.0,2404.0,22.4,NaN,NaN
571,200,2016-11-20 00:00:00,NaN,Natural,Natural,40.0,14.9,43.0,375,0.48,180,NaN,1845,700,1140,100.0,2396.0,22.4,29.1,NaN
572,228,2016-03-23 00:00:00,NaN,Crushed,Natural,40.0,14.7,43.0,375,0.48,180,Type SP901 Dos. 1.0 lit,1845,685,1160,180.0,2405.0,26.1,NaN,NaN


In [4]:
g30 = g30[['Cement O.P.C (Kg/m3)','Water Content (Kg/m3)','Fine Aggregate (Kg/m3)','Coarse Aggregate (Kg/m3)','Strength (N/mm2)             (7 days avg.)','Workability Slump (mm)']]
inputs = ['Cement O.P.C (Kg/m3)','Water Content (Kg/m3)','Fine Aggregate (Kg/m3)','Coarse Aggregate (Kg/m3)']
output1 = ['Strength (N/mm2)             (7 days avg.)']
output2 = ['Workability Slump (mm)']

In [5]:
g30.isnull().sum()

Cement O.P.C (Kg/m3)                          0
Water Content (Kg/m3)                         0
Fine Aggregate (Kg/m3)                        0
Coarse Aggregate (Kg/m3)                      0
Strength (N/mm2)             (7 days avg.)    6
Workability Slump (mm)                        3
dtype: int64

In [6]:
g30 = g30.dropna()

In [7]:
g30

,Cement O.P.C (Kg/m3),Water Content (Kg/m3),Fine Aggregate (Kg/m3),Coarse Aggregate (Kg/m3),Strength (N/mm2) (7 days avg.),Workability Slump (mm)
0,375,180,715,1070,37.6,180.0
1,375,180,730,1095,25.1,110.0
2,385,180,730,1095,39.2,230.0
3,365,185,780,1120,30.7,180.0
4,365,185,780,1120,27.0,180.0
...,...,...,...,...,...,...
569,375,165,690,1170,30.4,165.0
570,375,180,700,1140,22.4,100.0
571,375,180,700,1140,22.4,100.0
572,375,180,685,1160,26.1,180.0


In [8]:
X = g30[inputs]
Y1 = g30[output1]
Y2 = g30[output2]

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X,
                                                    Y1,
                                                    test_size=0.2,
                                                    random_state=0
                                                   )

In [10]:
# Fitting the model

from sklearn.ensemble import RandomForestRegressor
  
 # create regressor object
model = RandomForestRegressor(n_estimators = 50, random_state = 0, max_depth=20)
  
# fit the regressor with x and y data
model.fit(x_train, y_train)

C:\Users\Prasad\AppData\Local\Temp\ipykernel_2280\3111553141.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


RandomForestRegressor(max_depth=20, n_estimators=50, random_state=0)

In [33]:
import time
start = time.time()

print( model.predict(x.T))

end = time.time()
print("Compilation time:", end - start, "seconds")

[29.59585556]
Compilation time: 0.02918243408203125 seconds


D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [29]:
x1 = [380,186.2,672.21,1187.6]

In [31]:
x = pd.DataFrame(x1)

In [27]:
x

[380, 186.2, 672.21, 1187.6]

In [35]:
# prediction
y_pred = model.predict(x_test)
#RMSE
rmse = metrics.mean_squared_error(y_test, y_pred,squared=False)
print("predicted cs ", rmse)

predicted cs  3.657999230773657


## Workability slump estimation

In [12]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(X,
                                                    Y2,
                                                    test_size=0.2,
                                                    random_state=0
                                                   )

In [13]:
  
 # create regressor object
model1 = RandomForestRegressor(n_estimators = 50, random_state = 0, max_depth=20)
  
# fit the regressor with x and y data
model1.fit(x_train1, y_train1)

C:\Users\Prasad\AppData\Local\Temp\ipykernel_2280\1567117056.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model1.fit(x_train1, y_train1)


RandomForestRegressor(max_depth=20, n_estimators=50, random_state=0)

In [14]:
# prediction
y_pred1 = model1.predict(x_test1)
#RMSE
rmse1 = metrics.mean_squared_error(y_test1, y_pred1,squared=False)
print("Root mean squared error is ", rmse1)

Root mean squared error is  23.588649740563
